# タスク 2: Amazon Nova Lite による小さなファイルのテキスト要約


このノートブックでは、Amazon Bedrock API に小さなテキスト文字列を直接取り込み (Amazon Nova Lite モデルを使用)、入力テキストを要約するように指示します。入力テキストの長さがモデルのコンテキストサイズ制限内であれば、この方法を適用して、通話記録、会議記録、書籍、記事、ブログ投稿、およびその他の関連コンテンツを要約できます。

## タスク 2.1: 環境の設定

このタスクでは、環境を設定します。

In [ ]:
# Create a service client by name using the default session.
import json
import os
import boto3
import botocore

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))


## タスク 2.2: 要約するテキストを含むプロンプトを作成する

このタスクでは、基盤モデルがサポートする最大長よりもトークンの少ない、短いテキストを使用します。このラボのサンプル入力テキストとして、Amazon Bedrock について紹介している [AWS ブログ投稿](https://aws.amazon.com/jp/blogs/machine-learning/announcing-new-tools-for-building-with-generative-ai-on-aws/)の段落を使用します。

プロンプトは命令 `次のテキストの要約を入力してください` から始まります。

In [ ]:
prompt_data = """
次のテキストの要約を入力してください:

AWS はお客様からのフィードバックをすべて取り入れ、本日、AI21 Labs、Anthropic、Stability AI、Amazon の FM に API 経由でアクセスできるようにする新しいサービスである Amazon Bedrock を発表できることを嬉しく思います。
Bedrock は、FM を使用して AI ベースの生成アプリケーションを構築および拡張する最も簡単な方法であり、すべてのビルダーがアクセスできるようになります。
Bedrock は、スケーラブルで信頼性が高く、安全な AWS マネージドサービスを通じて、テキストと画像用のさまざまな強力な FM (本日発表する 2 つの新しい LLM で構成される Amazon の Titan FM を含む) にアクセスする機能を提供します。 
Bedrock のサーバーレス エクスペリエンスにより、お客様は、実行しようとしていることに適したモデルを簡単に見つけ、すぐに開始し、独自のデータを使用して FM をプライベートにカスタマイズし、使い慣れた AWS ツールと機能を使用してアプリケーションに簡単に統合してデプロイできます。
インフラストラクチャを管理する必要はありません
(さまざまなモデルをテストするための Experiments や、FM を大規模に管理するための Pipelines などの Amazon SageMaker ML 機能との統合を含む)。
"""

## タスク 2.3: プロンプトと推論パラメータを含むリクエストボディを作成する 

このタスクでは、上記のプロンプトと推論パラメータを使用してリクエストボディを作成します。

In [ ]:
# Nova Lite request body
body = json.dumps({
    "messages": [
        {
            "role": "user",
            "content": [{"text": prompt_data}]
        }
    ],
    "inferenceConfig": {
        "maxTokens": 2048,
        "temperature": 0,
        "topP": 0.9
    }
})

## タスク 2.4: Boto3 経由で基盤モデルを呼び出す

このタスクではリクエストパラメータ、`modelId`、`accept`、`contentType` を指定して Amazon Bedrock に API リクエストを送信します。指定したプロンプトに従い、Amazon Bedrock の基礎モデルが入力テキストを要約します。

### 出力生成の実行

デフォルトでは、Amazon Bedrock サービスは特定のプロンプトの要約全体を 1 つの出力で生成します。モデル出力に多くのトークンが含まれている場合、時間がかかる可能性があります。

In [ ]:
#model configuration and invoke the model
modelId = 'amazon.nova-lite-v1:0'
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"

try:

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    
    outputText = response_body.get('output').get('message').get('content')[0].get('text')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\nTo troubleshoot this issue please refer to the following resources.\nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

print(outputText)

### 出力生成のストリーミング

次に、Amazon Bedrock の invoke_model_with_response_stream API を使用してモデルの出力をストリーミングし、出力が生成されていくのに伴い、ユーザーがそれを使用できるようにする方法を探ります。この API は、出力全体を一度に生成するのではなく、ResponseStream を返します。これは、モデルからより小さい出力チャンクが生成されしだい、そのチャンクを送信します。これらのストリーミング出力は、連続的でわかりやすいビューで表示できます。

以下のコードを実行すると、raw のストリーミングレスポンス形式が表示されます。これはチャンクオブジェクトのリストで構成されています。各チャンクにはバイナリのペイロードが含まれており、これをデコードすると、テキストフラグメントとともに、トークン数や完了ステータスなどのメタデータが表示されます。例えば、Nova Lite の場合、各チャンクにテキストフラグメント、トークン数、停止理由の指標が含まれています。この raw 形式には、モデルがコンテンツを一度にまとめて生成するのではなく、単語ごと、またはフレーズごとに徐々に生成することが示されています。次のセルでは、これらのチャンクを処理して、生成されたコンテンツがより読みやすく、連続して表示されるようにします。

In [ ]:
#invoke model with response stream
modelId = 'amazon.nova-lite-v1:0'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')

# Collect only the first few chunks
output = []
for i, event in enumerate(stream):
    output.append(event)
    if i >= 4:  # Show only first 5 chunks
        print(f"... and {i} more chunks (truncated)")
        break

# Display truncated output
output

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
modelId = 'amazon.nova-lite-v1:0'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            if 'contentBlockDelta' in chunk_obj:
                text = chunk_obj['contentBlockDelta']['delta']['text']
                clear_output(wait=True)
                output.append(text)
                display_markdown(Markdown(''.join(output)))
                i+=1

これで、Boto3 SDK を使用して Amazon Bedrock API にアクセスする実験ができました。この SDK により、Bedrock 機能に対してプログラムによる基本的なアクセスが可能になります。この API を活用することで、2 つのユースケースを実装できました。1) AWS ニュースコンテンツのテキスト要約全体を一度に生成することと、2) 要約出力をチャンク単位でストリーミングしてインクリメンタル処理を行うことです。

### 自分でやってみよう
- プロンプトを特定のユースケースに合わせて変更し、さまざまなモデルの出力を評価します。
- トークンの長さを変えてみて、サービスのレイテンシーと応答性を理解します。
- さまざまなプロンプトエンジニアリングの原則を適用して、より良い出力を引き出します。

### クリーンアップ

このノートブックを完了しました。ラボの次の部分に進むために、以下を実行してください。

- このノートブックファイルを閉じて、**まとめ**に進みます。